# Addition of PV and local storage units at busses 3 and 4


In [35]:
using JuMP, HiGHS

# ------------------------------
# Data and Parameters
# ------------------------------

# Time periods
T = 1:8

# Scenarios
scenarios = 1:9

# Scenario probabilities (equal probabilities)
prob = Dict(s => 1/9 for s in scenarios)

# Wind availability at Bus 5 (stochastic, per scenario and time period)
W = Dict(
    (1,1) => 32.63, (1,2) => 31.40, (1,3) => 28.52, (1,4) => 31.40, (1,5) => 33.26, (1,6) => 33.43, (1,7) => 34.59, (1,8) => 34.06,
    (2,1) => 32.63, (2,2) => 31.40, (2,3) => 28.52, (2,4) => 31.40, (2,5) => 33.26, (2,6) => 33.43, (2,7) => 34.59, (2,8) => 34.06,
    (3,1) => 32.63, (3,2) => 31.40, (3,3) => 28.52, (3,4) => 31.40, (3,5) => 33.26, (3,6) => 33.43, (3,7) => 34.59, (3,8) => 34.06,
    
    (4,1) => 15.79, (4,2) => 13.04, (4,3) => 10.11, (4,4) => 10.33, (4,5) => 11.47, (4,6) => 12.69, (4,7) => 15.69, (4,8) => 16.73,
    (5,1) => 15.79, (5,2) => 13.04, (5,3) => 10.11, (5,4) => 10.33, (5,5) => 11.47, (5,6) => 12.69, (5,7) => 15.69, (5,8) => 16.73,
    (6,1) => 15.79, (6,2) => 13.04, (6,3) => 10.11, (6,4) => 10.33, (6,5) => 11.47, (6,6) => 12.69, (6,7) => 15.69, (6,8) => 16.73,

    (7,1) => 53.79, (7,2) => 54.13, (7,3) => 59.43, (7,4) => 66.39, (7,5) => 67.63, (7,6) => 64.22, (7,7) => 60.45, (7,8) => 55.51,
    (8,1) => 53.79, (8,2) => 54.13, (8,3) => 59.43, (8,4) => 66.39, (8,5) => 67.63, (8,6) => 64.22, (8,7) => 60.45, (8,8) => 55.51,
    (9,1) => 53.79, (9,2) => 54.13, (9,3) => 59.43, (9,4) => 66.39, (9,5) => 67.63, (9,6) => 64.22, (9,7) => 60.45, (9,8) => 55.51
)

# Dynamic load at Bus 1 (stochastic, per scenario and time period)
D = Dict(
    (1,1) => 5.0, (1,2) => 8.2, (1,3) => 20.5, (1,4) => 35.0, (1,5) => 42.8, (1,6) => 38.4, (1,7) => 25.1, (1,8) => 12.0,
    (2,1) => 10.0, (2,2) => 15.4, (2,3) => 30.7, (2,4) => 50.2, (2,5) => 61.3, (2,6) => 55.0, (2,7) => 35.7, (2,8) => 18.5,
    (3,1) => 15.0, (3,2) => 22.5, (3,3) => 42.1, (3,4) => 68.9, (3,5) => 82.5, (3,6) => 74.2, (3,7) => 48.6, (3,8) => 25.9,

    (4,1) => 5.0, (4,2) => 8.2, (4,3) => 20.5, (4,4) => 35.0, (4,5) => 42.8, (4,6) => 38.4, (4,7) => 25.1, (4,8) => 12.0,
    (5,1) => 10.0, (5,2) => 15.4, (5,3) => 30.7, (5,4) => 50.2, (5,5) => 61.3, (5,6) => 55.0, (5,7) => 35.7, (5,8) => 18.5,
    (6,1) => 15.0, (6,2) => 22.5, (6,3) => 42.1, (6,4) => 68.9, (6,5) => 82.5, (6,6) => 74.2, (6,7) => 48.6, (6,8) => 25.9,

    (7,1) => 5.0, (7,2) => 8.2, (7,3) => 20.5, (7,4) => 35.0, (7,5) => 42.8, (7,6) => 38.4, (7,7) => 25.1, (7,8) => 12.0,
    (8,1) => 10.0, (8,2) => 15.4, (8,3) => 30.7, (8,4) => 50.2, (8,5) => 61.3, (8,6) => 55.0, (8,7) => 35.7, (8,8) => 18.5,
    (9,1) => 15.0, (9,2) => 22.5, (9,3) => 42.1, (9,4) => 68.9, (9,5) => 82.5, (9,6) => 74.2, (9,7) => 48.6, (9,8) => 25.9
)

# Stochastic PV availability at Bus 3 (maximum available, per scenario and time period)
PV = Dict(
    (1,1) => 0.0,  (1,2) => 0.51,  (1,3) => 14.71, (1,4) => 43.31, (1,5) => 34.40, (1,6) => 6.39,  (1,7) => 0.09,  (1,8) => 0.0,
    (2,1) => 0.0,  (2,2) => 0.51,  (2,3) => 14.71, (2,4) => 43.31, (2,5) => 34.40, (2,6) => 6.39,  (2,7) => 0.09,  (2,8) => 0.0,
    (3,1) => 0.0,  (3,2) => 0.51,  (3,3) => 14.71, (3,4) => 43.31, (3,5) => 34.40, (3,6) => 6.39,  (3,7) => 0.09,  (3,8) => 0.0,

    (4,1) => 0.0,  (4,2) => 2.39,  (4,3) => 35.49, (4,4) => 68.73, (4,5) => 59.69, (4,6) => 19.08, (4,7) => 0.49,  (4,8) => 0.0,
    (5,1) => 0.0,  (5,2) => 2.39,  (5,3) => 35.49, (5,4) => 68.73, (5,5) => 59.69, (5,6) => 19.08, (5,7) => 0.49,  (5,8) => 0.0,
    (6,1) => 0.0,  (6,2) => 2.39,  (6,3) => 35.49, (6,4) => 68.73, (6,5) => 59.69, (6,6) => 19.08, (6,7) => 0.49,  (6,8) => 0.0,

    (7,1) => 0.0,  (7,2) => 0.04,  (7,3) => 2.66,  (7,4) => 12.62, (7,5) => 11.59, (7,6) => 1.30,  (7,7) => 0.01,  (7,8) => 0.0,
    (8,1) => 0.0,  (8,2) => 0.04,  (8,3) => 2.66,  (8,4) => 12.62, (8,5) => 11.59, (8,6) => 1.30,  (8,7) => 0.01,  (8,8) => 0.0,
    (9,1) => 0.0,  (9,2) => 0.04,  (9,3) => 2.66,  (9,4) => 12.62, (9,5) => 11.59, (9,6) => 1.30,  (9,7) => 0.01,  (9,8) => 0.0
)


# Conventional generation parameters at Bus 1
Gmax = 250.0   # Maximum generation capacity at Bus 1
c_g = 10.0     # Generation cost per unit
c_curt = 5.0 # Penalty cost per unit of wind curtailment 

# Penalty cost for load shedding 
c_ls = 100.0

# Ramping limits for conventional generation (per time period)
ramp_up = 50.0
ramp_down = 50.0

# Network parameters for DC load flow
X = 1.0  # Reactance 
Fmax = 250.0  # Flow limits on all lines


# ------------------------------
# Storage Parameters (for buses 3 and 4)
# ------------------------------

# Maximum storage energy (MWh)
E_max3 = 50.0
E_max4 = 50.0

# Maximum charging/discharging power (MW)
P_ch_max3 = 20.0
P_dis_max3 = 20.0
P_ch_max4 = 20.0
P_dis_max4 = 20.0

# Charging and discharging efficiencies
eta_ch = 0.95
eta_dis = 0.95

# Initial state-of-charge (MWh)
E_init3 = 0.0
E_init4 = 0.0

0.0

In [36]:
# ------------------------------
# Model Definition
# ------------------------------
model = Model(HiGHS.Optimizer)

# Conventional generation at Bus 1 (indexed by scenario and time)
@variable(model, g_conv[s in scenarios, t in T] >= 0, upper_bound = Gmax)

# Wind generation at Bus 5 (indexed by scenario and time)
@variable(model, g_wind[s in scenarios, t in T] >= 0)
@variable(model, c_w[s in scenarios, t in T] >= 0)

# PV generation at Bus 3 and Bus 4 (indexed by scenario and time)
@variable(model, g_PV3[s in scenarios, t in T] >= 0)
@variable(model, c_pv3[s in scenarios, t in T] >= 0)
@variable(model, g_PV4[s in scenarios, t in T] >= 0)
@variable(model, c_pv4[s in scenarios, t in T] >= 0)

# Storage variables for Bus 3:
@variable(model, E3[s in scenarios, t in T] >= 0)       # State-of-charge
@variable(model, p_ch3[s in scenarios, t in T] >= 0)    # Charging power
@variable(model, p_dis3[s in scenarios, t in T] >= 0)   # Discharging power

# Storage variables for Bus 4:
@variable(model, E4[s in scenarios, t in T] >= 0)       # State-of-charge
@variable(model, p_ch4[s in scenarios, t in T] >= 0)    # Charging power
@variable(model, p_dis4[s in scenarios, t in T] >= 0)   # Discharging power

# Network flows on each line (indexed by scenario and time)
@variable(model, f51[s in scenarios, t in T])
@variable(model, f12[s in scenarios, t in T])
@variable(model, f23[s in scenarios, t in T])
@variable(model, f24[s in scenarios, t in T])

# Voltage angles at buses 1 through 5 (indexed by scenario and time)
@variable(model, theta[bus in 1:5, s in scenarios, t in T])

# Fix reference angle at Bus 1 for all scenarios and time periods
for s in scenarios, t in T
    @constraint(model, theta[1,s,t] == 0)
end

@variable(model, LS3[s in scenarios, t in T] >= 0)
@variable(model, LS4[s in scenarios, t in T] >= 0)

# ------------------------------
# Objective Function
# ------------------------------

# The objective minimizes the cost of conventional generation and penalizes unused renewables.
@objective(model, Min, 
    sum(prob[s] * (c_g * g_conv[s,t] + c_curt * (c_w[s,t] + c_pv3[s,t] + c_pv4[s,t]) 
    + c_ls * (LS3[s,t] + LS4[s,t])) for s in scenarios, t in T))


# ------------------------------
# Constraints
# ------------------------------

# Wind generation constraint: wind used must not exceed available wind.
for s in scenarios, t in T
    @constraint(model, g_wind[s,t] + c_w[s,t] == W[(s,t)])

end

# PV generation constraints at Bus 3 and Bus 4:
for s in scenarios, t in T
    @constraint(model, g_PV3[s,t] + c_pv3[s,t] == PV[(s,t)])
    @constraint(model, g_PV4[s,t] + c_pv4[s,t] == PV[(s,t)])
end

# Storage charging/discharging limits and energy capacity constraints at Bus 3:
for s in scenarios, t in T
    @constraint(model, p_ch3[s,t] <= P_ch_max3)
    @constraint(model, p_dis3[s,t] <= P_dis_max3)
    @constraint(model, E3[s,t] <= E_max3)
end

# Storage charging/discharging limits and energy capacity constraints at Bus 4:
for s in scenarios, t in T
    @constraint(model, p_ch4[s,t] <= P_ch_max4)
    @constraint(model, p_dis4[s,t] <= P_dis_max4)
    @constraint(model, E4[s,t] <= E_max4)
end

# Storage dynamic constraints for Bus 3:
for s in scenarios
    # Initial period dynamics for Bus 3:
    @constraint(model, E3[s,1] == E_init3 + eta_ch * p_ch3[s,1] - (1/eta_dis) * p_dis3[s,1])
    # Subsequent periods:
    for t in T[2:end]
        @constraint(model, E3[s,t] == E3[s,t-1] + eta_ch * p_ch3[s,t] - (1/eta_dis) * p_dis3[s,t])
    end
end

# Storage dynamic constraints for Bus 4:
for s in scenarios
    # Initial period dynamics for Bus 4:
    @constraint(model, E4[s,1] == E_init4 + eta_ch * p_ch4[s,1] - (1/eta_dis) * p_dis4[s,1])
    # Subsequent periods:
    for t in T[2:end]
        @constraint(model, E4[s,t] == E4[s,t-1] + eta_ch * p_ch4[s,t] - (1/eta_dis) * p_dis4[s,t])
    end
end

# DC power flow equations and flow limits for each line, scenario, and time period:
for s in scenarios, t in T
    @constraint(model, f51[s,t] == (theta[5,s,t] - theta[1,s,t]) / X)
    @constraint(model, f12[s,t] == (theta[1,s,t] - theta[2,s,t]) / X)
    @constraint(model, f23[s,t] == (theta[2,s,t] - theta[3,s,t]) / X)
    @constraint(model, f24[s,t] == (theta[2,s,t] - theta[4,s,t]) / X)
    
    @constraint(model, f51[s,t] <= Fmax)
    @constraint(model, -f51[s,t] <= Fmax)
    @constraint(model, f12[s,t] <= Fmax)
    @constraint(model, -f12[s,t] <= Fmax)
    @constraint(model, f23[s,t] <= Fmax)
    @constraint(model, -f23[s,t] <= Fmax)
    @constraint(model, f24[s,t] <= Fmax)
    @constraint(model, -f24[s,t] <= Fmax)
end

# Power balance constraints at each bus for each scenario and time period:
for s in scenarios, t in T
    # Bus 5 (wind bus): Wind generation flows to Bus 1.
    @constraint(model, g_wind[s,t] - f51[s,t] == 0)
    
    # Bus 1: Conventional generation plus inflow from Bus 5 meets its own demand and supplies Bus 2.
    @constraint(model, g_conv[s,t] - D[(s,t)] + f51[s,t] - f12[s,t] == 0)
    
    # Bus 2: Receives power from Bus 1, meets its dynamic load, and sends power to Buses 3 and 4.
    @constraint(model, f12[s,t] - D[(s,t)] - f23[s,t] - f24[s,t] == 0)
    
    # Bus 3: Local PV generation at Bus 3 now contributes to meeting the fixed demand.
    @constraint(model, f23[s,t] + LS3[s,t] + g_PV3[s,t] - D[(s,t)] - p_ch3[s,t] + p_dis3[s,t] == 0)
    
    # Bus 4: Local PV generation at Bus 4 now contributes to meeting the fixed demand.
    @constraint(model, f24[s,t] + LS4[s,t] + g_PV4[s,t] - D[(s,t)] - p_ch4[s,t] + p_dis4[s,t] == 0)
end

# Ramping constraints for conventional generation at Bus 1 (for each scenario, between consecutive time periods):
for s in scenarios, t in T[2:end]
    @constraint(model, g_conv[s,t] - g_conv[s,t-1] <= ramp_up)
    @constraint(model, g_conv[s,t-1] - g_conv[s,t] <= ramp_down)
end

In [37]:
# ------------------------------
# Solve the Model
# ------------------------------

optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [9e-01, 1e+00]
  Cost   [6e-01, 1e+01]
  Bound  [2e+02, 2e+02]
  RHS    [1e-02, 2e+02]
Presolving model
558 rows, 1044 cols, 1926 nonzeros  0s
460 rows, 1009 cols, 1730 nonzeros  0s
Presolve : Reductions: rows 460(-1754); columns 1009(-719); elements 1730(-2752)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
        526     6.5105124694e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 526
Objective value     :  6.5105124694e+03
Relative P-D gap    :  1.1175706441e-15
HiGHS run time      :          0.01


In [38]:
# ------------------------------
# Output the Results
# ------------------------------

println("Overall Optimal Objective Value: ", objective_value(model))

# Print results per scenario and time period
for s in scenarios, t in T
    println("Scenario $s, Time Period $t:")
    println("  Conventional Generation (Bus 1): ", value(g_conv[s,t]))
    println("  Wind Generation Used (Bus 5): ", value(g_wind[s,t]))
    println("  Wind Curtailment (Bus 5): ", W[(s,t)] - value(g_wind[s,t]))
    println("  PV Generation at Bus 3: ", value(g_PV3[s,t]))
    println("  Storage at Bus 3: SoC = ", value(E3[s,t]),
            ", Charge = ", value(p_ch3[s,t]),
            ", Discharge = ", value(p_dis3[s,t]))
    println("  PV Generation at Bus 4: ", value(g_PV4[s,t]))
    println("  Storage at Bus 4: SoC = ", value(E4[s,t]),
            ", Charge = ", value(p_ch4[s,t]),
            ", Discharge = ", value(p_dis4[s,t]))
end

Overall Optimal Objective Value: 6510.512469412315
Scenario 1, Time Period 1:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 32.63
  Wind Curtailment (Bus 5): 0.0
  PV Generation at Bus 3: -0.0
  Storage at Bus 3: SoC = 11.598500000000003, Charge = 12.208947368421057, Discharge = 0.0
  PV Generation at Bus 4: -0.0
  Storage at Bus 4: SoC = 0.3999999999999989, Charge = 0.42105263157894623, Discharge = 0.0
Scenario 1, Time Period 2:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 31.4
  Wind Curtailment (Bus 5): 0.0
  PV Generation at Bus 3: 0.51
  Storage at Bus 3: SoC = 11.598500000000003, Charge = 0.0, Discharge = 0.0
  PV Generation at Bus 4: 0.51
  Storage at Bus 4: SoC = -0.0, Charge = 0.0, Discharge = 0.379999999999999
Scenario 1, Time Period 3:
  Conventional Generation (Bus 1): 14.73142499999999
  Wind Generation Used (Bus 5): 28.52
  Wind Curtailment (Bus 5): 0.0
  PV Generation at Bus 3: 14.71
  Storage at Bus 3: SoC = 1.778947368

In [39]:
println("Expected Conventional Generation: ", sum(prob[s] * sum(value(g_conv[s,t]) for t in T) for s in scenarios))
println("Expected Curtailment: ", sum(prob[s] * sum(value(c_w[s,t] + c_pv3[s,t] + c_pv4[s,t]) for t in T) for s in scenarios))
println("Expected Load Shedding: ", sum(prob[s] * sum(value(LS3[s,t] + LS4[s,t]) for t in T) for s in scenarios))

println("Expected Charging: ", sum(prob[s] * sum(value(p_ch3[s,t] + p_ch4[s,t]) for t in T) for s in scenarios))
println("Expected Discharging: ", sum(prob[s] * sum(value(p_dis3[s,t] + p_dis4[s,t]) for t in T) for s in scenarios))

Expected Conventional Generation: 650.8879136078982
Expected Curtailment: 0.3266666666666703
Expected Load Shedding: 0.0
Expected Charging: 58.80025352830093
Expected Discharging: 41.67567325373605
